In [36]:
#Script to reformat flux xls file for use in a stacked bar chart

#Created: 12.04.2016
#Last modified: 14.04.2016 Sorted data by abs_anthro value; added abs_anthro_perArea col
#               18.04.2016 Output carbon flux per surface area instead of unit area

In [1]:
import numpy as np
import pandas as pd
import collections
import os
import xlrd

In [45]:
molarmassC = 12.011 #g/mol

In [35]:
# Create dataframe from flux file

xl = pd.ExcelFile("/homel/cnangini/PROJECTS/CO2airsea/for_DataViz_complete.xls")
xl.sheet_names
#[u'Sheet1', u'Sheet2', u'Sheet3']
#[u'Sheet1']
df = xl.parse("Sheet1")
df.head(10)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,ORCA2,Unnamed: 5,Unnamed: 6,ORCA05,Unnamed: 8,Unnamed: 9,ORCA025,Unnamed: 11,Unnamed: 12
0,NaN,"Laruelle et al. (2013, 2014)",NaN,NaN,Surface\n(10^3 km^2),Air-sea flux\n(mol C m^-2 yr^-1),NaN,Surface\n(10^3 km^2),Air-sea flux\n(mol C m^-2 yr^-1),NaN,Surface\n(10^3 km^2),Air-sea flux\n(mol C m^-2 yr^-1),NaN
1,ID,MARCATS,CLASS,Mean Latitude (°N),NaN,Total,Anthropogenic,NaN,Total,Anthropogenic,NaN,Total,Anthropogenic
2,2,California Current,EBC,31.43,71.9735,0.258775,-0.192213,118.311,-0.33556,-0.35,197.109,-0.408655,-0.3447023
3,4,Peruvian Upwelling Current,EBC,-20.98,130.317,-0.0424662,-0.4052452,137.516,1.44461,-0.39,143.984,1.40228,-0.34623
4,19,Iberian Upwelling,EBC,47.39,174.844,-0.953769,-0.298491,251.093,-1.12506,-0.27,278.965,-0.850057,-0.235254
5,22,Moroccan Upwelling,EBC,22.46,111.796,0.192162,-0.211935,176.947,0.18122,-0.33,210.894,0.201397,-0.343572
6,24,Southern Western Africa,EBC,-27.99,307.138,-1.14521,-0.532412,299.792,0.428844,-0.59,306.152,0.413103,-0.578043
7,33,Leeuwin Current,EBC,-28.21,38.0426,-0.909239,-0.257277,81.4649,-2.05471,-0.6,110.191,-1.8733,-0.56869
8,27,Western Arabian Sea,Indian margins,17.08,20.4899,-0.062801,-0.200482,81.8175,0.347792,-0.31,104.031,0.470945,-0.309286
9,30,Eastern Arabian Sea,Indian margins,16.61,340.797,0.1413,-0.32113,323.5,0.209194,-0.3,355.181,0.285837,-0.254938


In [46]:
#model ORCA05 only
df05 = df.drop(['Unnamed: 0', 'ORCA2', 'Unnamed: 5', 'Unnamed: 6', 'ORCA025', 'Unnamed: 11', 'Unnamed: 12'], 1)
df05.head(5)

,Unnamed: 1,Unnamed: 2,Unnamed: 3,ORCA05,Unnamed: 8,Unnamed: 9
0,"Laruelle et al. (2013, 2014)",NaN,NaN,Surface\n(10^3 km^2),Air-sea flux\n(mol C m^-2 yr^-1),NaN
1,MARCATS,CLASS,Mean Latitude (°N),NaN,Total,Anthropogenic
2,California Current,EBC,31.43,118.311,-0.33556,-0.35
3,Peruvian Upwelling Current,EBC,-20.98,137.516,1.44461,-0.39
4,Iberian Upwelling,EBC,47.39,251.093,-1.12506,-0.27


In [47]:
#rename columns
df05.columns = ['MARCATS', 'RCLASS', 'LAT', 'AREA', 'TOTAL', 'ANTHRO']
df05.head(5)

,MARCATS,RCLASS,LAT,AREA,TOTAL,ANTHRO
0,"Laruelle et al. (2013, 2014)",NaN,NaN,Surface\n(10^3 km^2),Air-sea flux\n(mol C m^-2 yr^-1),NaN
1,MARCATS,CLASS,Mean Latitude (°N),NaN,Total,Anthropogenic
2,California Current,EBC,31.43,118.311,-0.33556,-0.35
3,Peruvian Upwelling Current,EBC,-20.98,137.516,1.44461,-0.39
4,Iberian Upwelling,EBC,47.39,251.093,-1.12506,-0.27


In [48]:
#Remove rows 0 and 1
df05 = df05.drop([0, 1], 0)

#Convert surface area from 10e3 km2 to m2
df05['AREA'] = df05['AREA'] * 10e9

#Convert fluxes from molC/(m2 yr) to g/(m2 yr)
df05['TOTAL'] = df05['TOTAL'] * molarmassC #* 10e-12
df05['ANTHRO'] = df05['ANTHRO'] * molarmassC

df05.head(5)

,MARCATS,RCLASS,LAT,AREA,TOTAL,ANTHRO
2,California Current,EBC,31.43,1.18311e+12,-4.030411,-4.20385
3,Peruvian Upwelling Current,EBC,-20.98,1.37516e+12,17.35121,-4.68429
4,Iberian Upwelling,EBC,47.39,2.51093e+12,-13.5131,-3.24297
5,Moroccan Upwelling,EBC,22.46,1.76947e+12,2.176633,-3.96363
6,Southern Western Africa,EBC,-27.99,2.99792e+12,5.150845,-7.08649


In [49]:
#Add column for abs value of ANTRHO for plotting purposes
df05['abs_anthro'] = -1 * df05['ANTHRO']
df05.head(5)

,MARCATS,RCLASS,LAT,AREA,TOTAL,ANTHRO,abs_anthro
2,California Current,EBC,31.43,1.18311e+12,-4.030411,-4.20385,4.20385
3,Peruvian Upwelling Current,EBC,-20.98,1.37516e+12,17.35121,-4.68429,4.68429
4,Iberian Upwelling,EBC,47.39,2.51093e+12,-13.5131,-3.24297,3.24297
5,Moroccan Upwelling,EBC,22.46,1.76947e+12,2.176633,-3.96363,3.96363
6,Southern Western Africa,EBC,-27.99,2.99792e+12,5.150845,-7.08649,7.08649


In [50]:
#Add columns for flux per surface area, expressed in Tg (10e12 g)

df05['total_perArea'] = df05['TOTAL'] * df05['AREA'] * 10e-12
df05['anthro_perArea'] = df05['ANTHRO'] * df05['AREA'] * 10e-12
df05['abs_anthro_perArea'] = df05['abs_anthro'] * df05['AREA'] * 10e-12
df05.head(5)

,MARCATS,RCLASS,LAT,AREA,TOTAL,ANTHRO,abs_anthro,total_perArea,anthro_perArea,abs_anthro_perArea
2,California Current,EBC,31.43,1.18311e+12,-4.030411,-4.20385,4.20385,-47.6842,-49.73617,49.73617
3,Peruvian Upwelling Current,EBC,-20.98,1.37516e+12,17.35121,-4.68429,4.68429,238.6069,-64.41648,64.41648
4,Iberian Upwelling,EBC,47.39,2.51093e+12,-13.5131,-3.24297,3.24297,-339.3044,-81.42871,81.42871
5,Moroccan Upwelling,EBC,22.46,1.76947e+12,2.176633,-3.96363,3.96363,38.51488,-70.13524,70.13524
6,Southern Western Africa,EBC,-27.99,2.99792e+12,5.150845,-7.08649,7.08649,154.4182,-212.4473,212.4473


In [51]:
#Sort by abs_anthro
#df_stats.sort('MtCO2', ascending=False).head(50)
df05 = df05.sort('abs_anthro', ascending=False)
df05.head(10)

,MARCATS,RCLASS,LAT,AREA,TOTAL,ANTHRO,abs_anthro,total_perArea,anthro_perArea,abs_anthro_perArea
24,Southern Greenland,Polar,64.75,2.3764e+12,-40.24754,-10.32946,10.32946,-956.4425,-245.4693,245.4693
28,Antarctic Shelves,Polar,-71.76,2.462e+13,-10.86007,-8.28759,8.28759,-2673.749,-2040.405,2040.405
7,Leeuwin Current,EBC,-28.21,8.14649e+11,-24.67912,-7.2066,7.2066,-201.0482,-58.70849,58.70849
25,Norwegian Basin,Polar,67.94,1.40691e+12,-34.51085,-7.2066,7.2066,-485.5365,-101.3904,101.3904
6,Southern Western Africa,EBC,-27.99,2.99792e+12,5.150845,-7.08649,7.08649,154.4182,-212.4473,212.4473
43,Florida Upwelling,WBC,40.62,7.84725e+12,-25.08629,-6.72616,6.72616,-1968.584,-527.8186,527.8186
14,North Eastern Atlantic,Marginal sea,56.69,1.02002e+13,-25.98147,-6.36583,6.36583,-2650.162,-649.3274,649.3274
44,Agulhas Current,WBC,-28.37,1.89285e+12,-14.42497,-6.36583,6.36583,-273.0431,-120.4956,120.4956
33,New Zealand,Subpolar,-45.15,2.62966e+12,-14.76188,-6.24572,6.24572,-388.1872,-164.2412,164.2412
20,Sea of Japan,Marginal sea,41.93,2.33304e+12,-36.91629,-6.12561,6.12561,-861.2718,-142.9129,142.9129


In [52]:
#============================
# PART I - STACKED BAR CHART
# Build tsv file
#============================

#Pivot table so that MARCATS col becomes first row with values corresponding to abs_anthro
#http://stackoverflow.com/questions/29942167/transposing-one-column-in-python-pandas-with-the-simplest-index-possible
#dc.pivot(columns='disease_header', index='medicine', values='disease').reset_index()

df05_piv = df05.pivot(columns='MARCATS', index='RCLASS', values='abs_anthro').reset_index()
#df05_piv = df05_piv.drop(['CLASS'], 1)
df05_piv

MARCATS,RCLASS,Agulhas Current,Antarctic Shelves,Baltic Sea,Barents & Kara Sea,Bay of Bengal,Black Sea,Brazilian Current,California Current,Canadian Archipeloagos,...,Southern America,Southern Australia,Southern Greenland,Southern Western Africa,Tropical Eastern Atlantic,Tropical Eastern Indian,Tropical Eastern Pacific,Tropical Western Atlantic,Tropical Western Indian,Western Arabian Sea
0,EBC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.20385,NaN,...,NaN,NaN,NaN,7.08649,NaN,NaN,NaN,NaN,NaN,NaN
1,Indian margins,NaN,NaN,NaN,NaN,3.72341,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2.4022,NaN,NaN,NaN,3.72341
2,Marginal sea,NaN,NaN,0.12011,NaN,NaN,2.16198,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Polar,NaN,8.28759,NaN,1.32121,NaN,NaN,NaN,NaN,1.08099,...,NaN,NaN,10.32946,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Subpolar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.52506,3.24297,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Tropical,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2.28209,NaN,4.32396,3.12286,1.92176,NaN
6,WBC,6.36583,NaN,NaN,NaN,NaN,NaN,4.08374,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
#Replace NaN with 0
df05_piv = df05_piv.replace(np.nan,0, regex=True)
df05_piv

MARCATS,RCLASS,Agulhas Current,Antarctic Shelves,Baltic Sea,Barents & Kara Sea,Bay of Bengal,Black Sea,Brazilian Current,California Current,Canadian Archipeloagos,...,Southern America,Southern Australia,Southern Greenland,Southern Western Africa,Tropical Eastern Atlantic,Tropical Eastern Indian,Tropical Eastern Pacific,Tropical Western Atlantic,Tropical Western Indian,Western Arabian Sea
0,EBC,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,4.20385,0.00000,...,0.00000,0.00000,0.00000,7.08649,0.00000,0.0000,0.00000,0.00000,0.00000,0.00000
1,Indian margins,0.00000,0.00000,0.00000,0.00000,3.72341,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,2.4022,0.00000,0.00000,0.00000,3.72341
2,Marginal sea,0.00000,0.00000,0.12011,0.00000,0.00000,2.16198,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.0000,0.00000,0.00000,0.00000,0.00000
3,Polar,0.00000,8.28759,0.00000,1.32121,0.00000,0.00000,0.00000,0.00000,1.08099,...,0.00000,0.00000,10.32946,0.00000,0.00000,0.0000,0.00000,0.00000,0.00000,0.00000
4,Subpolar,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,5.52506,3.24297,0.00000,0.00000,0.00000,0.0000,0.00000,0.00000,0.00000,0.00000
5,Tropical,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,2.28209,0.0000,4.32396,3.12286,1.92176,0.00000
6,WBC,6.36583,0.00000,0.00000,0.00000,0.00000,0.00000,4.08374,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.0000,0.00000,0.00000,0.00000,0.00000


In [54]:
df05_piv.columns.name

'MARCATS'

In [55]:
df05_piv.columns.name = ''
df05_piv

,RCLASS,Agulhas Current,Antarctic Shelves,Baltic Sea,Barents & Kara Sea,Bay of Bengal,Black Sea,Brazilian Current,California Current,Canadian Archipeloagos,...,Southern America,Southern Australia,Southern Greenland,Southern Western Africa,Tropical Eastern Atlantic,Tropical Eastern Indian,Tropical Eastern Pacific,Tropical Western Atlantic,Tropical Western Indian,Western Arabian Sea
0,EBC,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,4.20385,0.00000,...,0.00000,0.00000,0.00000,7.08649,0.00000,0.0000,0.00000,0.00000,0.00000,0.00000
1,Indian margins,0.00000,0.00000,0.00000,0.00000,3.72341,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,2.4022,0.00000,0.00000,0.00000,3.72341
2,Marginal sea,0.00000,0.00000,0.12011,0.00000,0.00000,2.16198,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.0000,0.00000,0.00000,0.00000,0.00000
3,Polar,0.00000,8.28759,0.00000,1.32121,0.00000,0.00000,0.00000,0.00000,1.08099,...,0.00000,0.00000,10.32946,0.00000,0.00000,0.0000,0.00000,0.00000,0.00000,0.00000
4,Subpolar,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,5.52506,3.24297,0.00000,0.00000,0.00000,0.0000,0.00000,0.00000,0.00000,0.00000
5,Tropical,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,2.28209,0.0000,4.32396,3.12286,1.92176,0.00000
6,WBC,6.36583,0.00000,0.00000,0.00000,0.00000,0.00000,4.08374,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.0000,0.00000,0.00000,0.00000,0.00000


In [15]:
#Save to tsv for stacked bar chart ( abs anthro flux in Tg/[m2 yr] )
#df05_piv.to_csv('/homel/cnangini/PROJECTS/CO2airsea/orca05_absAnthro.tsv', float_format='%.2f', sep='\t')

In [56]:
#Create table for abs_anthro per Surface Area

#Pivot orig table
df05_piv_perArea = df05.pivot(columns='MARCATS', index='RCLASS', values='abs_anthro_perArea').reset_index()

#Replace NaN with 0
df05_piv = df05_piv.replace(np.nan,0, regex=True)
df05_piv_perArea

MARCATS,RCLASS,Agulhas Current,Antarctic Shelves,Baltic Sea,Barents & Kara Sea,Bay of Bengal,Black Sea,Brazilian Current,California Current,Canadian Archipeloagos,...,Southern America,Southern Australia,Southern Greenland,Southern Western Africa,Tropical Eastern Atlantic,Tropical Eastern Indian,Tropical Eastern Pacific,Tropical Western Atlantic,Tropical Western Indian,Western Arabian Sea
0,EBC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,49.73617,NaN,...,NaN,NaN,NaN,212.4473,NaN,NaN,NaN,NaN,NaN,NaN
1,Indian margins,NaN,NaN,NaN,NaN,73.359,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,174.5549,NaN,NaN,NaN,30.46401
2,Marginal sea,NaN,NaN,3.894074,NaN,NaN,28.22076,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Polar,NaN,2040.405,NaN,219.1623,NaN,NaN,NaN,NaN,111.8933,...,NaN,NaN,245.4693,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Subpolar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,622.2765,127.1076,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Tropical,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,51.40248,NaN,65.86213,149.5641,8.925441,NaN
6,WBC,120.4956,NaN,NaN,NaN,NaN,NaN,193.7559,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [66]:
#Replace NaN with 0
df05_piv_perArea = df05_piv_perArea.replace(np.nan,0, regex=True)
df05_piv_perArea

MARCATS,RCLASS,Agulhas Current,Antarctic Shelves,Baltic Sea,Barents & Kara Sea,Bay of Bengal,Black Sea,Brazilian Current,California Current,Canadian Archipeloagos,...,Southern America,Southern Australia,Southern Greenland,Southern Western Africa,Tropical Eastern Atlantic,Tropical Eastern Indian,Tropical Eastern Pacific,Tropical Western Atlantic,Tropical Western Indian,Western Arabian Sea
0,EBC,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,49.73617,0.000000,...,0.000000,0.000000,0.000000,212.447301,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000
1,Indian margins,0.000000,0.000000,0.000000,0.000000,73.358996,0.000000,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00000,174.554902,0.000000,0.000000,0.000000,30.46401
2,Marginal sea,0.000000,0.000000,3.894074,0.000000,0.000000,28.220757,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000
3,Polar,0.000000,2040.404658,0.000000,219.162315,0.000000,0.000000,0.000000,0.00000,111.893275,...,0.000000,0.000000,245.469287,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000
4,Subpolar,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,622.276458,127.107561,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000
5,Tropical,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.000000,51.40248,0.000000,65.862126,149.564071,8.925441,0.00000
6,WBC,120.495613,0.000000,0.000000,0.000000,0.000000,0.000000,193.755903,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000


In [67]:
#Save to tsv for stacked bar chart ( abs anthro flux per Surface Area in Tg/yr )
df05_piv_perArea.to_csv('/homel/cnangini/PROJECTS/CO2airsea/orca05_absAnthroPerArea_TgPerYr.tsv', float_format='%.2f', sep='\t')

In [19]:
#============================
# PART II - SANKEY DIAGRAM
# Build json file
#============================

In [57]:
#Get sources list
sources = np.unique(df05.iloc[:,1].values) #RCLASS COL
sources

array([u'EBC', u'Indian margins', u'Marginal sea', u'Polar', u'Subpolar',
       u'Tropical', u'WBC'], dtype=object)

In [59]:
#Get regions list
regions = df05.iloc[:,0].values #MARCAT col
regions

array([u'Southern Greenland', u'Antarctic Shelves', u'Leeuwin Current',
       u'Norwegian Basin', u'Southern Western Africa',
       u'Florida Upwelling', u'North Eastern Atlantic', u'Agulhas Current',
       u'New Zealand', u'Sea of Japan', u'Eastern Austrialian Current',
       u'Southern America', u'North Eastern Pacific',
       u'China Sea & Kuroshio', u'Peruvian Upwelling Current',
       u'Sea of Okhotsk', u'Tropical Eastern Pacific',
       u'California Current', u'Brazilian Current', u'Moroccan Upwelling',
       u'Gulf of Mexico', u'Western Arabian Sea', u'Bay of Bengal',
       u'Sea of Labrador', u'Carribean Sea', u'Eastern Arabian Sea',
       u'Mediterranean Sea', u'Iberian Upwelling', u'Southern Australia',
       u'Northern Greenland', u'Tropical Western Atlantic',
       u'North Western Pacific', u'Northern Australia',
       u'Tropical Eastern Indian', u'South East Asia',
       u'Tropical Eastern Atlantic', u'Black Sea',
       u'Tropical Western Indian', u'Red Sea'

In [60]:
nodes = sources.tolist() + regions.tolist()
nodes

[u'EBC',
 u'Indian margins',
 u'Marginal sea',
 u'Polar',
 u'Subpolar',
 u'Tropical',
 u'WBC',
 u'Southern Greenland',
 u'Antarctic Shelves',
 u'Leeuwin Current',
 u'Norwegian Basin',
 u'Southern Western Africa',
 u'Florida Upwelling',
 u'North Eastern Atlantic',
 u'Agulhas Current',
 u'New Zealand',
 u'Sea of Japan',
 u'Eastern Austrialian Current',
 u'Southern America',
 u'North Eastern Pacific',
 u'China Sea & Kuroshio',
 u'Peruvian Upwelling Current',
 u'Sea of Okhotsk',
 u'Tropical Eastern Pacific',
 u'California Current',
 u'Brazilian Current',
 u'Moroccan Upwelling',
 u'Gulf of Mexico',
 u'Western Arabian Sea',
 u'Bay of Bengal',
 u'Sea of Labrador',
 u'Carribean Sea',
 u'Eastern Arabian Sea',
 u'Mediterranean Sea',
 u'Iberian Upwelling',
 u'Southern Australia',
 u'Northern Greenland',
 u'Tropical Western Atlantic',
 u'North Western Pacific',
 u'Northern Australia',
 u'Tropical Eastern Indian',
 u'South East Asia',
 u'Tropical Eastern Atlantic',
 u'Black Sea',
 u'Tropical Wester

In [62]:
#df05.iloc['California Current']
df05.iloc[0]['abs_anthro']

10.32946

In [72]:
df05.head(5)

,MARCATS,RCLASS,LAT,AREA,TOTAL,ANTHRO,abs_anthro,total_perArea,anthro_perArea,abs_anthro_perArea
24,Southern Greenland,Polar,64.75,2.3764e+12,-40.24754,-10.32946,10.32946,-956.4425,-245.4693,245.4693
28,Antarctic Shelves,Polar,-71.76,2.462e+13,-10.86007,-8.28759,8.28759,-2673.749,-2040.405,2040.405
7,Leeuwin Current,EBC,-28.21,8.14649e+11,-24.67912,-7.2066,7.2066,-201.0482,-58.70849,58.70849
25,Norwegian Basin,Polar,67.94,1.40691e+12,-34.51085,-7.2066,7.2066,-485.5365,-101.3904,101.3904
6,Southern Western Africa,EBC,-27.99,2.99792e+12,5.150845,-7.08649,7.08649,154.4182,-212.4473,212.4473


In [73]:
file = open('sankey_orca05_anthroflux_perSurfaceArea.json', 'w')

file.write('{\n')
file.write('"nodes": [\n')
for node in nodes:
    file.write('{"name": "%s"},\n' %(node))
# remove last comma
file.seek(-2, os.SEEK_END)
file.truncate()
file.write('\n],\n')

file.write('"links": [\n')
idx = 0;
#for source in sources:
for region in regions:
    source = df05.iloc[idx]['RCLASS']
    #for region in regions:
    print idx
    print source
    print region
    print df05.iloc[idx]['abs_anthro_perArea']
    file.write('{"source": "%s", "target": "%s", "value": "%.2f"},\n' %(source, region, df05.iloc[idx]['abs_anthro_perArea']))
    idx = idx + 1
# remove last comma
file.seek(-2, os.SEEK_END)
file.truncate()
file.write('\n]\n')
file.write('}\n')

file.close()

0
Polar
Southern Greenland
245.46928744
1
Polar
Antarctic Shelves
2040.404658
2
EBC
Leeuwin Current
58.708494834
3
Polar
Norwegian Basin
101.39037606
4
EBC
Southern Western Africa
212.447301008
5
WBC
Florida Upwelling
527.8185906
6
Marginal sea
North Eastern Atlantic
649.32739166
7
WBC
Agulhas Current
120.495613155
8
Subpolar
New Zealand
164.241200552
9
Marginal sea
Sea of Japan
142.912931544
10
WBC
Eastern Austrialian Current
58.525218985
11
Subpolar
Southern America
622.27645768
12
Subpolar
North Eastern Pacific
214.48138788
13
WBC
China Sea & Kuroshio
611.66197665
14
EBC
Peruvian Upwelling Current
64.416482364
15
Marginal sea
Sea of Okhotsk
403.365364956
16
Tropical
Tropical Eastern Pacific
65.862126324
17
EBC
California Current
49.736169735
18
WBC
Brazilian Current
193.755902918
19
EBC
Moroccan Upwelling
70.135243761
20
Marginal sea
Gulf of Mexico
180.414060096
21
Indian margins
Western Arabian Sea
30.4640097675
22
Indian margins
Bay of Bengal
73.358996161
23
Subpolar
Sea of Labrad